In [ ]:
%matplotlib inline

# Create bike track table from ways, with unwind order and lengths 🚲 🚲 🚲

This notebook assembles a table of bike tracks from already derived ways, determining track components and paths (unwind order) and track lengths. Tracks with the same name are merged.  

**Derived biketracks is dropped and re-inserted!**

Created on:  2016-12-07  
Last update: 2017-01-22  
Contact: michael.szell@gmail.com (Michael Szell)

## Preliminaries

### Parameters

In [ ]:
cityname = "vienna"

### Imports

In [ ]:
# preliminaries
from __future__ import unicode_literals
import sys
import re
import csv
import os
import math
import pprint
pp = pprint.PrettyPrinter(indent=4)
import requests
import gzip
from collections import defaultdict
import time
import datetime
import numpy as np
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry

import osmnx as ox
import networkx as nx
import json
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from functools import partial
import pyproj
from scipy import spatial
from haversine import haversine

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt

### DB Connection

In [ ]:
client = MongoClient()
db_raw = client[cityname+'_raw']
ways_raw = db_raw['ways']
nodes_raw = db_raw['nodes']
db_derived = client[cityname+'_derived']
biketracks = db_derived['biketracks']
ways_derived = db_derived['ways']

### Functions

In [ ]:
def isbiketrack(wayinfo):
    if "properties" in wayinfo:
        try:
            if "highway" in wayinfo["properties"]:
                if wayinfo["properties"]["highway"]["highway"] == "cycleway":
                    return True
            if "bicycle" in wayinfo["properties"]:
                if wayinfo["properties"]["bicycle"]["bicycle"] == "designated":
                    return True    
            if "cycleway" in wayinfo["properties"]:
                if wayinfo["properties"]["cycleway"]["cycleway"] == "track":
                    return True
        except: 
            pass
    return False
        
def assembleBikeTrack(ways_raw, ways_derived, nodes_raw, wayid, biketrackname, drawit = False, debug = False):
    oxelements = []
    elemnodesint = set()
    elemnodesint_real = set()
    elemways = []
    elemnodes = []
    nodesdict = {}
    biketrackwidth = 0
        
    if biketrackname: # merge same-named bike tracks
        cursor = ways_raw.find({"tags.name.name": biketrackname})
    else: # just the way
        cursor = ways_raw.find({"_id": wayid})
        
        
    for i,way in enumerate(cursor):
        wayinfo = ways_derived.find_one({"_id":int(way["_id"])})
        if isbiketrack(wayinfo):
            if not biketrackwidth and "width" in way["tags"]:
                try:
                    biketrackwidth = float(way["tags"]["width"]["width"])
                except: # check for width in feet
                    if str(way["tags"]["width"]["width"]).find("'") > -1:
                        feetwidth = float(re.sub("[^0-9]", "", way["tags"]["width"]["width"]))
                        biketrackwidth = feetwidth * 0.3048      
            for n in way["nodes"]:
                elemnodesint.add(int(n))

    for n in elemnodesint:
        for nd in nodes_raw.find({"_id": n}):
            elemnodesint_real.add(int(nd["_id"]))
            elemnodes.append({"id": int(nd["_id"]), "lat": nd["loc"]["coordinates"][1], "lon": nd["loc"]["coordinates"][0], "type": "node"})
            nodesdict[str(int(nd["_id"]))] = {"lat": nd["loc"]["coordinates"][1], "lon": nd["loc"]["coordinates"][0]}

    biketracklength = 0
    if biketrackname: # merge same-named bike tracks
        cursor = ways_raw.find({"tags.name.name": biketrackname})                               
    else: # just the way
        cursor = ways_raw.find({"_id": wayid})
        
    for i,way in enumerate(cursor):
        wayinfo = ways_derived.find_one({"_id":int(way["_id"])})
        if isbiketrack(wayinfo):
            waylength = wayinfo["properties_derived"]["length"]
            biketracklength += waylength
            elemways.append({"id": int(way["_id"]), "nodes":[int(way["nodes"][k]) for k in range(len(way["nodes"])) if int(way["nodes"][k]) in elemnodesint_real], "tags": way["tags"], "type": "way"})

    oxelements = [{"elements": elemnodes + elemways}]
    if debug:
        # Check if nodes are missing
        if len(elemnodesint_real) < len(elemnodesint):
            print(str(len(elemnodesint)-len(elemnodesint_real)) + " nodes are missing.")
        print("oxelements:")
        pp.pprint(oxelements)
        
    G = ox.create_graph(oxelements, retain_all=True)
    if drawit:
        fig, ax = ox.plot_graph(G)
    G = nx.Graph(G)
    G = G.to_undirected()
    ccs = list(nx.connected_component_subgraphs(G))

    components = []
    for c in range(len(ccs)):
        deglist = np.array(list((ccs[c].degree_iter())))
        endptindices = np.where(deglist[:, 1] == 1)
        # Look which of the endpoints lies most western, take that as the source for DFS traversal
        west = float('inf')
        source = deglist[0, 0]
        for i in list(endptindices[0]):
            westthis = nodesdict[str(deglist[i, 0])]["lon"]
            if westthis < west:
                source = deglist[i, 0]
                west = westthis

        component = []
        dfsedges = list(nx.dfs_edges(ccs[c], source))
        nend = dfsedges[0][0]
        path = [str(nend)]
        for e in dfsedges:
            if e[0] == nend: # path grows
                path.append(str(e[1]))
            else: # new path
                component.append(path)
                path = [str(e[0]), str(e[1])]
            nend = e[1]    
        component.append(path) # last path
        components.append(component)
    
    output = {"tags":{"name":biketrackname, "length": biketracklength, "width": biketrackwidth}, "components":components, "nodes":nodesdict, "ways":[elemways[k]["id"] for k in range(len(elemways))]}
    return output

In [ ]:
biketracks.drop()  
biketracks_error = []
biketracknamesused = set()
cursor = ways_raw.find({"$or": [{"tags.highway.highway":"cycleway"},{"tags.bicycle.bicycle":"designated"}, {"tags.cycleway.cycleway":"track"}]})
bar = pyprind.ProgBar(cursor.count(), bar_char='█', update_interval=1)
for i,way in enumerate(cursor):
    try:
        if "name" in way["tags"]:
            thisname = way["tags"]["name"]["name"]
        else:
            thisname = ""
        if not thisname or (thisname and thisname not in biketracknamesused):
            res = assembleBikeTrack(ways_raw, ways_derived, nodes_raw, way["_id"], thisname)
            res["_id"] = i+1
            biketracks.update_one({"_id":res["_id"]}, {"$set":res}, upsert=True)
            if thisname:
                biketracknamesused.add(thisname)
    except:
        biketracks_error.append(way["_id"])
    bar.update(item_id = i)
    
with open("logs/" + cityname +"_biketrackserror.txt", "w") as f: 
    for s in biketracks_error:
        f.write("%s\n" % s)
        
with open("citydata/" + cityname +"_biketracknames.txt", "w") as f: 
    for s in biketracknamesused:
        f.write("%s\n" % s)